In [1]:
%load_ext cudf.pandas
# pandas API is now GPU accelerated

import pandas as pd
# Check it is running on GPU
print(pd)

# Geopandas is a geo dataframe
# import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

<module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>


In [2]:
import sys
import os
print(sys.executable)

/home/kim/anaconda3/envs/rapids-24.02/bin/python


In [2]:
# import cupy as cp
# print(cp.cuda.runtime.getDeviceCount())
# conda activate rapids-24.02

# Conda environment

- python = 3.10 
- cuda-version = 12.0 (Roger is 12.2)

# Load cleaned (preprocesses) data

Crashed previously. This was resolved by taking .conda-bedmap kernel

In [4]:
all = pd.read_csv("/home/kim/data/bedmap/bedmap123_preprocessed.csv")
# might take 2-3 mins
all.shape

(65238004, 11)

In [5]:
# drop first two columns
all.drop(all.iloc[:, 0:2], inplace = True, axis = 1)
all

,lon,lat,x,y,s,t,b,b_inferred,source
0,-163.000000,-78.788330,-3.572549e+05,-1.168528e+06,197.00,415.00,-218.00,True,BEDMAP1_1966-2000_AIR_BM1.csv
1,-177.100000,-78.976670,-6.077581e+04,-1.199732e+06,46.00,329.00,-283.00,True,BEDMAP1_1966-2000_AIR_BM1.csv
2,-84.810810,-77.516990,-1.355903e+06,1.231389e+05,529.00,2213.00,-1684.00,True,BEDMAP1_1966-2000_AIR_BM1.csv
3,-84.760420,-77.563450,-1.350710e+06,1.238651e+05,514.00,2213.00,-1699.00,True,BEDMAP1_1966-2000_AIR_BM1.csv
4,-84.703970,-77.615780,-1.344861e+06,1.246650e+05,505.00,2180.00,-1675.00,True,BEDMAP1_1966-2000_AIR_BM1.csv
...,...,...,...,...,...,...,...,...,...
65237999,171.568072,-77.409951,2.013652e+05,-1.358404e+06,-14.30,213.01,-227.31,False,UTIG_2010_ICECAP_AIR_BM3.csv
65238000,171.568076,-77.409746,2.013684e+05,-1.358427e+06,-13.86,212.07,-225.93,False,UTIG_2010_ICECAP_AIR_BM3.csv
65238001,171.568079,-77.409540,2.013717e+05,-1.358449e+06,-14.60,210.89,-225.50,False,UTIG_2010_ICECAP_AIR_BM3.csv
65238002,171.568083,-77.409331,2.013750e+05,-1.358472e+06,-15.03,213.22,-228.25,False,UTIG_2010_ICECAP_AIR_BM3.csv


## Pruning function

In [6]:
def prune_all(df, res, min_count):
    zero_offset = res/2
    
    # grid at highest resolution
    df["x_highestres"] = (np.round((df.x - zero_offset) / res) * res) + zero_offset  
    df["y_highestres"] = (np.round((df.y - zero_offset) / res) * res) + zero_offset  

    # Make categocial column based on grid cell index
    df["id_highestres"] = df["x_highestres"].astype(int).astype(str) + "_" + df["y_highestres"].astype(int).astype(str)

    # Drop above columns to avoid kernel crashing?!
    # df = df.drop(columns = [["x_highestres"], ["y_highestres"]])
    # print(df.shape)

    # b is arbitrary: just count rows
    grid_count_series = df.groupby(by = "id_highestres")["b"].count()
    # make dataframe
    grid_count_df = pd.DataFrame(grid_count_series)
    # rename from "b" to "grid_count"
    grid_count_df = grid_count_df.rename(columns = {"b": "grid_count"})

    df_count_merged = df.merge(grid_count_df, on = "id_highestres")
    print("Full dataframe:", df_count_merged.shape[0])

    df_pruned = df_count_merged.drop(df_count_merged.loc[(df_count_merged["grid_count"] < min_count)].index)
    print("Pruned dataframe:", df_pruned.shape[0])
    print("Percentage of measurements excluded due to count threshold:", np.round(df_pruned.shape[0]/df_count_merged.shape[0], 3), "with min_count", min_count, "and min resolution", res)

    return df_pruned

In [7]:
min_count = 2

# We now start with 62.5
res_list = [62.5, 125, 250]
# Initialise

rmse_list = []
mae_list = []
rate_list = []

# Smallest res as input
# Pruning takes around 4 minutes
all_pruned = prune_all(all, res = res_list[0], min_count = min_count)
all_pruned.shape

Full dataframe: 65238004
cub/detail/detect_cuda_runtime.cuh(39): warning: cuda_runtime_api.h: [jitify] File not found
../util_type.cuh(42): warning: cuda.h: [jitify] File not found
cupy_jitify_exercise(10): warning: cooperative_groups.h: [jitify] File not found
cupy_jitify_exercise(11): warning: cooperative_groups/memcpy_async.h: [jitify] File not found
Pruned dataframe: 61229863
Percentage of measurements excluded due to count threshold: 0.939 with min_count 2 and min resolution 62.5


(61229863, 13)

In [8]:
# remove columns we do not need
all_pruned.drop(all_pruned.iloc[:, 9:13], inplace = True, axis = 1)

# delete so we have space
del all

# Gridding function

In [9]:
def gridding_error(df, zero_offset, res):
    # x grid values for 500m grid
    # 250 offset of 0 is first deducted
    df["x_500"] = (np.round((df.x - zero_offset) / res) * res) + zero_offset  
    df["y_500"] = (np.round((df.y - zero_offset) / res) * res) + zero_offset  

    # Make categocial column based on grid cell index
    df["id_500"] = df["x_500"].astype(int).astype(str) + "_" + df["y_500"].astype(int).astype(str)

    grid_b_series = df.groupby(by = "id_500")["b"].mean()
    print("debug 1")
    grid_b_df = pd.DataFrame(grid_b_series)
    grid_b_df = grid_b_df.rename(columns = {"b": "grid_mean_b"})
    # count
    grid_count = df.groupby(by = "id_500")["b"].count()
    # add to df
    grid_b_df["count"] = np.array(grid_count)
    print("debug 2")

    df_merged = df.merge(grid_b_df, on = "id_500")
    print("debug 3")
    df_merged["error"] = df_merged["grid_mean_b"] - df_merged["b"]

    # Calculate rate of grid cells without measurements (in square)
    x_gridvalues = np.arange(np.min(df_merged.x_500), np.max(df_merged.x_500) + 1, res)
    y_gridvalues = np.arange(np.min(df_merged.y_500), np.max(df_merged.y_500) + 1, res)
    rate = grid_b_df.shape[0]/(x_gridvalues.shape[0] * y_gridvalues.shape[0])
      
    return  df_merged, rate

In [ ]:
# REWRITE THE GRIDDING FUNCTION
def gridding_error(df, zero_offset, res):
    # x grid values for 500m grid
    # 250 offset of 0 is first deducted
    df["x_500"] = (np.round((df.x - zero_offset) / res) * res) + zero_offset  
    df["y_500"] = (np.round((df.y - zero_offset) / res) * res) + zero_offset  

    # Make categocial column based on grid cell index
    df["id_500"] = df["x_500"].astype(int).astype(str) + "_" + df["y_500"].astype(int).astype(str)

    grid_b_series = df.groupby(by = "id_500")["b"].mean()
    print("debug 1")
    grid_b_df = pd.DataFrame(grid_b_series)
    grid_b_df = grid_b_df.rename(columns = {"b": "grid_mean_b"})
    # count
    grid_count = df.groupby(by = "id_500")["b"].count()
    # add to df
    grid_b_df["count"] = np.array(grid_count)
    print("debug 2")

    df_merged = df.merge(grid_b_df, on = "id_500")
    print("debug 3")
    df_merged["error"] = df_merged["grid_mean_b"] - df_merged["b"]

    # Calculate rate of grid cells without measurements (in square)
    x_gridvalues = np.arange(np.min(df_merged.x_500), np.max(df_merged.x_500) + 1, res)
    y_gridvalues = np.arange(np.min(df_merged.y_500), np.max(df_merged.y_500) + 1, res)
    rate = grid_b_df.shape[0]/(x_gridvalues.shape[0] * y_gridvalues.shape[0])
      
    return  df_merged, rate

# Run

In [10]:
# We now start with 62.5
res_list = [62.5, 125, 250, 500, 1000, 2000]
# Initialise

rmse_list = []
mae_list = []
rate_list = []

# Takes around 4 min per res
for i in res_list:
    zo = i/2
    resso = i
    # run over all
    gridded_df, rate = gridding_error(all_pruned, zero_offset = zo, res = resso)
    rmse = np.sqrt(np.mean(np.square(gridded_df["error"])))
    print(rmse)
    mae = np.mean(np.abs(gridded_df["error"]))
    # Append
    rmse_list.append(rmse)
    mae_list.append(mae)
    rate_list.append(rate)

    print("Resolution ", i, "completed.")

debug 1
debug 2
debug 3
20.7442339219287
Resolution  62.5 completed.
debug 1
debug 2


: 

In [ ]:
min_count = 2

# We now start with 62.5
res_list2 = [500, 1000, 2000]
# Initialise

rmse_list2 = []
mae_list2 = []
rate_list2 = []

# Smallest res as input
# all_pruned = prune_all(all, res = res_list[0], min_count = min_count)

for i in res_list2:
    zo = i/2
    res = i
    # run over all
    gridded_df, rate = gridding_error(all_pruned, zero_offset = zo, res = res)
    rmse = np.sqrt(np.mean(np.square(gridded_df[gridded_df["count"] > min_count]["error"])))
    print(rmse)
    mae = np.mean(np.abs(gridded_df[gridded_df["count"] > min_count]["error"]))
    # Append
    rmse_list2.append(rmse)
    mae_list2.append(mae)
    rate_list2.append(rate)

    print("Resolution ", i, "completed.")

Run in 2 portions or otherwise import cupy as np

In [ ]:
res_list_combined = np.concatenate((res_list, res_list2))
rmse_list_combined = np.concatenate((rmse_list, rmse_list2))
mae_list_combined = np.concatenate((mae_list, mae_list2))
rate_list_combined = np.concatenate((rate_list, rate_list2))

# Save results

In [ ]:
data = {'resolution': res_list,
        'rmse': rmse_list,
        'mae': mae_list,
        'rate': rate_list}

experiment_results_all = pd.DataFrame(data)

experiment_results_all.to_csv('results/experiment_results_all.csv', index=False)

## Plots

In [ ]:
plt.plot(res_list, rmse_list, "--bo")

plt.xlabel("grid cell size (resolution)")
plt.ylabel("RMSE")
plt.ylim(bottom = 0)
plt.title("Gridding error (RMSE) by resolution for Antarctica (61 M measurements)")
plt.axvline(x = 500, ls='--', color = "lightgrey")

In [ ]:
plt.plot(res_list, mae_list, "--bo")

plt.xlabel("grid cell size (resolution)")
plt.ylabel("MAE")
plt.title("Gridding error (MAE) by resolution for Antarctica (61 M measurements)")
plt.ylim(bottom = 0)
plt.axvline(x = 500, ls='--', color = "lightgrey")

In [ ]:
gridded_df.sort_values('error')

## Dome A

In [ ]:
domea = all[(all["x"] > 898750) & (all["x"] < 1498750) & (all["y"] > 250) & (all["y"] < 600250)]
# 1.8 M values in dome A region

# The domain lies between [898,750, 1,498,750] projection
# x coordinate (in meters), and [250, 600,250] projection y coordinate (in meters).

np.mean(domea["t"])

In [ ]:
domea.to_csv("/home/kim/data/bedmap/bedmap123_preprocessed_DomeA.csv")

In [ ]:
plt.plot(res_list, da_rmse_list, "--bo", label = "Dome A")
plt.plot(res_list, rmse_list, "--go", label = "Antarctica")
plt.xlabel("grid resolution")
plt.ylabel("RMSE")
plt.title("Gridding error (RMSE) by resolution")
plt.legend()
plt.ylim(bottom = 0)
plt.axvline(x = 500, ls='--', color = "lightgrey")
plt.show()

In [ ]:
plt.plot(res_list, da_mae_list, "--bo", label = "Dome A")
plt.plot(res_list, mae_list, "--go", label = "Antarctica")
plt.xlabel("grid resolution")
plt.ylabel("MAE")
plt.title("Gridding error (MAE) by resolution")
plt.legend()
plt.ylim(bottom = 0)
plt.axvline(x = 500, ls='--', color = "lightgrey")
plt.show()